In [1]:
import requests
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import time
import os

from sentence_transformers import SentenceTransformer

I:\Anaconda\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Scrapping danych i zapis

## Utworzenie dataframe i słownika dat
utworzenie pustego dataframe politicians_df z potrzebnymi kolumnami i stworzenie słownika ze wszystkimi datami i posiedzeniami sejmu

In [3]:
politicians_df = pd.DataFrame(columns=["politician", "political_club", "utterance", "sejm_session", "date"])

# KADENCJA X
gov_meetings_dict = {
    "2023-11-13": [1],
    "2023-11-14": [1],
    "2023-11-21": [1],
    "2023-11-22": [1],
    "2023-11-28": [1],
    "2023-11-29": [1],
    "2023-12-06": [1],
    "2023-12-07": [1],
    "2023-12-11": [1],
    "2023-12-12": [1],
    "2023-12-19": [1],
    "2023-12-20": [1],
    "2023-12-21": [1],
    "2024-01-16": [2],
    "2024-01-17": [2],
    "2024-01-18": [3],
    "2024-01-25": [4],
    "2024-01-26": [4],
    "2024-02-07": [5],
    "2024-02-08": [5],
    "2024-02-09": [5],
    "2024-02-21": [6],
    "2024-02-22": [6],
    "2024-03-06": [7],
    "2024-03-07": [7],
    "2024-03-08": [7],
    "2024-03-20": [8],
    "2024-03-21": [8],
    "2024-04-10": [9],
    "2024-04-11": [9],
    "2024-04-12": [9],
    "2024-04-24": [10],
    "2024-04-25": [10],
    "2024-04-26": [10],
    "2024-05-08": [11],
    "2024-05-09": [11],
    "2024-05-15": [11],
    "2024-05-22": [12],
    "2024-05-23": [12],
    "2024-06-12": [13],
    "2024-06-13": [13],
    "2024-06-14": [13],
    "2024-06-26": [14],
    "2024-06-27": [14],
    "2024-06-28": [14],
    "2024-07-11": [15],
    "2024-07-12": [15],
    "2024-07-23": [16],
    "2024-07-24": [16],
    "2024-07-25": [16],
    "2024-07-26": [16],
    "2024-09-11": [17],
    "2024-09-12": [17],
    "2024-09-13": [17],
    "2024-09-25": [18],
    "2024-09-26": [18],
    "2024-09-27": [18],
    "2024-10-01": [18],
    "2024-10-09": [19],
    "2024-10-10": [19],
    "2024-10-11": [19],
    "2024-10-16": [20],
    "2024-10-17": [20],
    "2024-10-18": [20],
    "2024-11-06": [21],
    "2024-11-07": [21],
    "2024-11-08": [21],
    "2024-11-19": [22],
    "2024-11-20": [22],
    "2024-11-21": [22],
    "2024-11-22": [22],
    "2024-11-27": [22],
    "2024-12-04": [23],
    "2024-12-05": [23],
    "2024-12-06": [24],
    "2024-12-18": [25],
    "2024-12-19": [25],
    "2024-12-20": [25],
    "2025-01-08": [26],
    "2025-01-09": [26],
    "2025-01-10": [26],
    "2025-01-22": [27],
    "2025-01-23": [27],
    "2025-01-24": [27]
}


In [4]:
politicians_df = pd.DataFrame(columns=["utterance"])

# KADENCJA IX
gov_meetings_dict = {
    "2019-11-12": [1],
    "2019-11-13": [1],
    "2019-11-19": [1],
    "2019-11-21": [1],
    
    "2019-12-12": [2],
    "2019-12-19": [2],
    "2019-12-20": [2],
    
    "2020-01-08": [3],
    "2020-01-09": [3],
    "2020-01-10": [3],
    
    "2020-01-22": [4],
    "2020-01-23": [4],
    
    "2020-02-12": [5],
    "2020-02-13": [5],

    "2020-02-14": [6],

    "2020-03-02": [7],

    "2020-03-26": [8],
    "2020-03-27": [8],

    "2020-03-31": [9],
    "2020-04-03": [9],
    "2020-04-06": [9],
    "2020-04-07": [9],
    "2020-04-08": [9],

    "2020-04-15": [10],
    "2020-04-16": [10],

    "2020-04-29": [11],
    "2020-04-30": [11],
    "2020-05-05": [11],
    "2020-05-06": [11],
    "2020-05-07": [11],
    "2020-05-12": [11],
    "2020-05-14": [11],

    "2020-05-27": [12],
    "2020-05-28": [12],
    "2020-06-02": [12],
    "2020-06-04": [12],

    "2020-06-19": [13],

    "2020-07-15": [14],
    "2020-07-16": [14],

    "2020-07-22": [15],
    "2020-07-23": [15],
    "2020-07-24": [15],

    "2020-08-14": [16],

    "2020-09-16": [17],
    "2020-09-17": [17],

    "2020-10-07": [18],

    "2020-10-20": [19],
    "2020-10-21": [19],
    "2020-10-22": [19],

    "2020-10-27": [20],
    "2020-10-28": [20],

    "2020-11-18": [21],
    "2020-11-19": [21],
    "2020-11-27": [21],
    "2020-11-28": [21],

    "2020-12-09": [22],
    "2020-12-10": [22],

    "2020-12-15": [23],
    "2020-12-16": [23],

    "2020-12-17": [24],

    "2021-01-20": [25],
    "2021-01-21": [25],

    "2021-02-24": [26],
    "2021-02-25": [26],

    "2021-03-16": [27],
    "2021-03-17": [27],
    "2021-03-30": [27],

    "2021-04-14": [28],
    "2021-04-15": [28],
    "2021-04-20": [28],

    "2021-05-04": [29],

    "2021-05-19": [30],
    "2021-05-20": [30],
    "2021-05-21": [30],
    "2021-05-28": [30],

    "2021-06-15": [31],

    "2021-06-16": [32],

    "2021-06-23": [33],
    "2021-06-24": [33],

    "2021-07-07": [34],
    "2021-07-08": [34],

    "2021-07-21": [35],
    "2021-07-22": [35],
    "2021-07-23": [35],
    
    "2021-08-11": [36],
    "2021-09-06": [36],
    "2021-09-15": [36],

    "2021-09-15": [37],
    "2021-09-16": [37],
    "2021-09-17": [37],

    "2021-09-29": [38],
    "2021-09-30": [38],
    "2021-10-01": [38],

    "2021-10-13": [39],
    "2021-10-14": [39],

    "2021-10-28": [40],
    "2021-10-29": [40],

    "2021-11-09": [41],

    "2021-11-16": [42],
    "2021-11-17": [42],

    "2021-11-30": [43],
    "2021-12-01": [43],
    "2021-12-02": [43],
    "2021-12-08": [43],
    "2021-12-09": [43],

    "2021-12-14": [44],
    "2021-12-15": [44],

    "2021-12-17": [45],

    "2022-01-12": [46],
    "2022-01-13": [46],

    "2022-01-26": [47],
    "2022-01-27": [47],
    "2022-02-01": [47],

    "2022-02-08": [48],
    "2022-02-09": [48],

    "2022-02-23": [49],
    "2022-02-24": [49],
    "2022-03-03": [49],

    "2022-03-08": [50],
    "2022-03-09": [50],
    "2022-03-11": [50],
    "2022-03-12": [50],

    "2022-03-23": [51],
    "2022-03-24": [51],

    "2022-04-06": [52],
    "2022-04-07": [52],
    "2022-04-13": [52],

    "2022-04-27": [53],
    "2022-04-28": [53],

    "2022-05-11": [54],
    "2022-05-12": [54],

    "2022-05-25": [55],
    "2022-05-26": [55],

    "2022-06-08": [56],
    "2022-06-09": [56],

    "2022-06-22": [57],
    "2022-06-23": [57],

    "2022-07-06": [58],
    "2022-07-07": [58],

    "2022-07-20": [59],
    "2022-07-21": [59],
    "2022-07-22": [59],
    "2022-08-05": [59],

    "2022-09-02": [60],

    "2022-09-14": [61],
    "2022-09-15": [61],
    "2022-09-16": [61],

    "2022-09-28": [62],
    "2022-09-29": [62],

    "2022-10-05": [63],
    "2022-10-06": [63],
    "2022-10-07": [63],
    "2022-10-20": [63],
    
    "2022-10-26": [64],
    "2022-10-27": [64],

    "2022-11-03": [65],
    "2022-11-04": [65],

    "2022-11-15": [66],
    "2022-11-16": [66],

    "2022-11-30": [67],
    "2022-12-01": [67],

    "2022-12-13": [68],
    "2022-12-14": [68],
    "2022-12-15": [68],

    "2022-12-15": [69],

    "2023-01-11": [70],
    "2023-01-12": [70],
    "2023-01-13": [70],

    "2023-01-25": [71],
    "2023-01-26": [71],

    "2023-02-07": [72],
    "2023-02-08": [72],
    "2023-02-09": [72],

    "2023-03-07": [73],
    "2023-03-08": [73],
    "2023-03-09": [73],

    "2023-04-12": [74],
    "2023-04-13": [74],
    "2023-04-14": [74],

    "2023-05-09": [75],

    "2023-05-24": [76],
    "2023-05-25": [76],
    "2023-05-26": [76],

    "2023-06-13": [77],
    "2023-06-14": [77],
    "2023-06-15": [77],
    "2023-06-16": [77],

    "2023-07-06": [78],
    "2023-07-07": [78],

    "2023-07-11": [79],
    "2023-07-12": [79],
    "2023-07-13": [79],
    
    "2023-07-28": [80],

    "2023-08-16": [81],
    "2023-08-17": [81],
    "2023-08-30": [81]
}


## Scrapping danych przy użyciu sejmowego API
- przeiterowanie po każdej dacie i numerze posiedzenia sejmu,
- stworzenie URL do API sejmu zawierającego konkretną datę oraz posiedzenie i pobranie każdej wypowiedzi z danego zebrania
- wyjęcie z 'h2' polityka (politician)
- wyjęcie zmiędzy paragrafów 'p' wypowiedzi (utterance)
- zapisanie wszystkiego do politicians_df

In [5]:
for name, key in gov_meetings_dict.items():
    for number in key:
        i = 1
        start_time = time.time()
        while True:
            url = f"https://api.sejm.gov.pl/sejm/term9/proceedings/{number}/{name}/transcripts/{i}"
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html')
        
            element = soup.find('h2', class_='mowca')
            if element is not None:
                politician = element.text.strip()[:-1]
            else:
                print(f'Koniec wypowiedzi. Odnotowano: {i} przemówień')
                break
        
            utterance_elements = []
            for p in soup.find_all('p')[1:]:
                utterance_elements.append(p.text.strip())
            utterance = "".join(utterance_elements).strip()
        
            nowy_wiersz = pd.DataFrame({"utterance": [utterance]})
            politicians_df = pd.concat([politicians_df, nowy_wiersz], ignore_index=True)
        
            i = i+1
        end_time = time.time()
        print(f'Czas posiedzenia nr {number} o dacie {name}: {end_time-start_time:.2f} sekundy \n')

Koniec wypowiedzi. Odnotowano: 47 przemówień
Czas posiedzenia nr 1 o dacie 2019-11-12: 5.80 sekundy 

Koniec wypowiedzi. Odnotowano: 10 przemówień
Czas posiedzenia nr 1 o dacie 2019-11-13: 1.51 sekundy 

Koniec wypowiedzi. Odnotowano: 292 przemówień
Czas posiedzenia nr 1 o dacie 2019-11-19: 30.92 sekundy 

Koniec wypowiedzi. Odnotowano: 137 przemówień
Czas posiedzenia nr 1 o dacie 2019-11-21: 14.84 sekundy 

Koniec wypowiedzi. Odnotowano: 191 przemówień
Czas posiedzenia nr 2 o dacie 2019-12-12: 19.88 sekundy 

Koniec wypowiedzi. Odnotowano: 218 przemówień
Czas posiedzenia nr 2 o dacie 2019-12-19: 21.88 sekundy 

Koniec wypowiedzi. Odnotowano: 149 przemówień
Czas posiedzenia nr 2 o dacie 2019-12-20: 15.48 sekundy 

Koniec wypowiedzi. Odnotowano: 268 przemówień
Czas posiedzenia nr 3 o dacie 2020-01-08: 28.08 sekundy 

Koniec wypowiedzi. Odnotowano: 227 przemówień
Czas posiedzenia nr 3 o dacie 2020-01-09: 22.91 sekundy 

Koniec wypowiedzi. Odnotowano: 83 przemówień
Czas posiedzenia nr 3 o

In [8]:
df_filtered = politicians_df[~politicians_df['utterance'].str.contains(r'\bślubu\w*', case=False, regex=True)]

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

def count_tokens(text):
    return len(tokenizer.encode(text, truncation=False))

In [19]:
# Upewnij się, że pracujesz na kopii df_filtered (żeby nie było widoku)
df_filtered = df_filtered.copy()

# Dodaj kolumnę token_count używając .loc (bezpiecznie)
df_filtered.loc[:, "token_count"] = df_filtered["utterance"].apply(count_tokens)

# Filtruj po tokenach i stwórz nową kopię
df_filtered2 = df_filtered[df_filtered["token_count"] <= 400].copy()

# Usuń kolumnę pomocniczą
df_filtered2.drop(columns=["token_count"], inplace=True)


## Sprawdzenie poprawności dataframe
różne informacje: pierwsza wartość, próbka, shape itp.

In [5]:
politicians_df.iloc[0,0]

'Prezydent Rzeczypospolitej Polskiej Andrzej Duda'

In [6]:
politicians_df.sample().iloc[0,1]

'Szanowna Pani Marszałek! Szanowna Izbo! Zanim zostałem posłem, byłem radnym \r\nmiasta Częstochowy. To jest pierwsze miasto w Polsce, które wprowadziło in \r\nvitro w 2012 r. W 2015 r. był taki moment, że radni Częstochowy chcieli \r\nzaktualizować ten program, zwiększyć kwotę przyznawaną parom i klub Lewicy i \r\nPlatformy był za, a klub PiS-u był przeciw.W 2015 r. to głosy naszego częstochowskiego klubu, klubu mieszkańców \r\nCzęstochowy, zdecydowały o tym, że do tej pory w Częstochowie funkcjonuje in \r\nvitro i urodziło się 77 dzieci.Cieszę się, że historia zatacza koło, bo wtedy w Częstochowie mieliśmy duży \r\ndylemat przez to, że mieszkańcy, osoby zameldowane w Częstochowie mogą \r\nkorzystać z tego programu, a mieszkańcy sąsiednich gmin (Dzwonek) jak \r\nBlachownia, Koniecpol czy Lubliniec - nie. Cieszę się, że mogę to dzisiaj \r\nnaprawić i razem z państwem zagłosować za in vitro. Dziękuję bardzo. \r\n(Oklaski)'

In [20]:
df_filtered2.shape

(23884, 1)

In [9]:
politicians_df.groupby('politician').size().sort_values(ascending=False)

politician
Poseł Witold Tumanowicz                                                                                                       360
Poseł Jarosław Sachajko                                                                                                       322
Poseł Zbigniew Bogucki                                                                                                        205
Poseł Tadeusz Tomaszewski                                                                                                     199
Poseł Grzegorz Braun                                                                                                          182
                                                                                                                             ... 
Poseł Zbigniew Chmielowiec (tekst niewygłoszony)                                                                                1
Poseł Sprawozdawca Alicja Chybicka                                             

## Dodanie danych o partiach politycznych
wykorzystanie sejmowego API do stworzenia słownika zawierającego partie polityczne i ich posłów, a następnie przeiterowanie po datasecie politicians_df aby nadać politykom ich kluby

In [2]:
url_politicians = "https://api.sejm.gov.pl/sejm/term10/MP"
page_politicians = requests.get(url_politicians)
political_parties_data = page_politicians.json()

In [12]:
club_dict = {}

for person in political_parties_data:
    club = person.get("club", "Brak klubu")
    name = person.get("firstLastName", "Nieznany")

    # Dodajemy do słownika
    if club in club_dict:
        club_dict[club].append(name)
    else:
        club_dict[club] = [name]

for club, names in club_dict.items():
    print(f"{club}: {', '.join(names)}\n")

PiS: Andrzej Adamczyk, Adam Andruszkiewicz, Waldemar Andzel, Dorota Arciszewska-Mielewczyk, Iwona Ewa Arent, Marek Ast, Piotr Babinetz, Ryszard Bartosik, Barbara Bartuś, Mariusz Błaszczak, Rafał Bochenek, Jacek Bogucki, Zbigniew Bogucki, Joanna Borowiak, Kamil Bortniczuk, Bożena Borys-Szopa, Waldemar Buda, Lidia Burzyńska, Zbigniew Chmielowiec, Artur Chojecki, Kazimierz Bogusław Choma, Dominika Chorosińska, Tadeusz Chrzan, Anna Ewa Cicholska, Krzysztof Ciecióra, Janusz Cieszyński, Michał Cieślak, Krzysztof Czarnecki, Witold Wojciech Czarnecki, Przemysław Czarnek, Arkadiusz Czartoryski, Anita Czerwińska, Katarzyna Czochara, Władysław Dajczak, Anna Dąbrowska-Banaszek, Zbigniew Dolata, Bartłomiej Dorywalski, Przemysław Drabek, Elżbieta Duda, Michał Paweł Dworczyk, Jan Michał Dziedziczak, Magdalena Filipek-Sobczak, Radosław Fogiel, Andrzej Gawron, Grzegorz Gaża, Anna Gembicka, Szymon Giżyński, Piotr Gliński, Małgorzata Golińska, Kazimierz Gołojuch, Robert Gontarz, Mariusz Gosek, Małgorzata

In [13]:
politicians_df["political_club"] = ""

In [22]:
club_dict = {}

def assign_political_club(row):
    politician_name = row["politician"]

    for club, names in club_dict.items():
        if any(name in politician_name for name in names):  # Szukamy nazwiska w całym stringu
            return club
    return "Nieznana partia"

politicians_df["political_club"] = politicians_df.apply(assign_political_club, axis=1)

In [35]:
politicians_df.sample()

,politician,utterance,sejm_session,date,political_club
8799,Poseł Adam Krzemiński,Panie Marszałku! Wysoka Izbo! Szanowna Pani Mi...,17,2024-09-12,KO


## Zapis dataframe do excela
plik z rozszerzeniem .xlsx

In [12]:
#politicians_df.to_excel('politycy.xlsx', index=False)

# DLA IX KADENCJI
#politicians_df.to_excel('politycy.xlsx', index=False)
df_filtered.to_excel('politycy_IX_kadencja.xlsx', index=False)

# Operowanie na zescrappowanych danych

## Przygotowanie datasetu z excela
- Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv
- Oczyszczenie datasetu (wypowiedzi - utterance) z niepotrzebnych afixów "\r" "\n"

In [22]:
#IX kadencja
#dataset_politycy = pd.read_csv('politycy_IX_kadencja.csv', encoding='utf-8', sep=';')
#dataset_politycy = dataset_politycy.dropna(subset=["utterance"])

In [16]:
dataset_politycy = pd.read_csv('politycy.csv', encoding='utf-8', sep=';')
dataset_politycy = dataset_politycy.dropna(subset=["utterance", "politician", "sejm_session", "date", "political_club"])

In [23]:
dataset_politycy['utterance'] = dataset_politycy['utterance'].str.replace(r'[\n\r]', '', regex=True)

In [24]:
dataset_politycy['utterance'][9003]

'Koło Polska 2050, dwa tysiące pięćdziesiąt.Szanowny Panie Marszałku! Wysoka Izbo! Przedstawiam stanowisko Polski 2050 wobec projektu ustawy o zmianie ustawy - Kodeks karny.Wasze pisane na kolanie ustawy są równie ogromnym zagrożeniem dla swobód obywatelskich i stanu praworządności w Polsce, jak działania Al-Kaidy dla pokoju na Bliskim Wschodzie. Jeśli chodzi o standardy ochrony prawnej, wypychacie nasze państwo prawa z serca demokratycznej Europy w paszczę wschodniego lewiatana.Mam pytanie do posłów wnioskodawców ze Zjednoczonej Prawicy. Czy minister Ziobro miał proroczy sen, a jego doradcy z Ordo Iuris niczym wróżki zębuszki przepowiedzieli, że wiosną naród upomni się o swoje brutalnie wyrwane prawa i wyrzuci go z ministerialnego gmachu? Po to wam te zapisy jak z podręcznika dla dyktatorów? Po to tworzycie zapisy prawa karnego, które wyjmują władzę wykonawczą spod jakiejkolwiek kontroli? Aby stawiać zarzuty kobietom aresztowanym podczas demonstracji, nieprzytomnym, przewiezionym do a

In [25]:
#IX kadencja
#dataset_politycy.to_excel('politycy_IX_kadencja_czysty.xlsx', index=False)

In [31]:
dataset_politycy.to_excel('politycy_czysty.xlsx', index=False)

## Podzielenie wypowiedzi na mniejsze segmenty (niekonieczne)
- Wykonanie na każdym wierszu funkcji podzielenia i pofiltrowania
- Podział wypowiedzi na zdania, punkty graniczne to między innymi kropka, przecinek, znak zapytania itp.
- Pofiltrowanie aby w datasecie zostały TYLKO zdania zawierające więcej niż 10 wyrazów

In [10]:
# poucinanie wypowiedzi na pojedyncze zdania i odrzucenie tych które są krótsze niż 15 słów

import re

def split_and_filter(row):
    #sentences = re.split(r'(?<=[.!?])\s+', row['utterance'])
    sentences = re.split(r'(?<=[.!?])\s*(?=[A-ZĄĆĘŁŃÓŚŹŻ])', row['utterance'])
    filtered_sentences = [s for s in sentences if len(s.split()) >= 15]
    return pd.DataFrame({'politician': row['politician'], 'utterance': filtered_sentences, 'political_club': row['political_club']})

dataset_filtered_politycy = pd.concat(dataset_politycy.apply(split_and_filter, axis=1).tolist(), ignore_index=True)
dataset_filtered_politycy.to_excel('politycy_filtered.xlsx', index=False)

# Model AI oraz embeddings

## Przygotowanie datasetu z excela
Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv

In [5]:
#dataset_filtered_politycy = pd.read_csv('politycy_filtered.csv', encoding='utf-8', sep=';')
#dataset_filtered_politycy = dataset_filtered_politycy.dropna(subset=["politician", "utterance", 'political_club'])

dataset_filtered_politycy = pd.read_csv('politycy_czysty.csv', encoding='utf-8', sep=';')
dataset_filtered_politycy = dataset_filtered_politycy.dropna(subset=["politician", "utterance", 'political_club'])

## Załadowanie odpowiedniego modelu AI oraz stworzenie funkcji
- funkcja get_embedding odpowiedzialna za stworzenie embeddingu na tekście
- funkcja get_similarity porównująca podobieństwo między embeddingami przy użyciu cosine similarity 

In [2]:
#model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#model = SentenceTransformer('BAAI/bge-multilingual-gemma2')
#model = SentenceTransformer('Alibaba-NLP/gte-multilingual-base')
#model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')

model = SentenceTransformer("polbert-sentence-model")

In [3]:
# Funkcja do konwersji tekstu na wektor (embedding)
def get_embedding(text):
    return model.encode(text)


def get_similarity(user_input, df_embeddings):
    user_embedding = get_embedding(user_input)
    
    similarities = []
    for index, row in df_embeddings.iterrows():
        politician_embedding = np.array(row['embedding'])
        
        similarity = cosine_similarity([user_embedding], [politician_embedding])[0][0]
        similarities.append((similarity, row['politician'], row['utterance']))

    similarities.sort(reverse=True, key=lambda x: x[0])
    top_n = 4
    avg_similarity = np.mean([sim for sim, _, _ in similarities[:top_n]])
    
    return avg_similarity, similarities[:top_n]


def get_similarity_politician(user_input, df_dataset):
    user_embedding = get_embedding(user_input)
    
    df_dataset["similarity"] = df_dataset["embedding"].apply(lambda emb: cosine_similarity([user_embedding], [emb])[0][0])
    df_sorted = df_dataset.sort_values(by="similarity", ascending=False)

    top_n = 3
    top_similar = df_sorted.head(top_n)

    return top_similar[["similarity", "politician", "political_club", "utterance"]].values.tolist()

## Sprawdzenie liczby tokenów dla każdej z wypowiedzi sejmowych

In [22]:
from transformers import AutoModel
#model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('dkleczek/bert-base-polish-cased-v1')
print(model.config)
print(model.config.max_position_embeddings)

BertConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 60000
}

512


In [10]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print(model)
print("max_seq_length:", model.max_seq_length)
model.max_seq_length = 512
print("max_seq_length:", model.max_seq_length)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
max_seq_length: 128
max_seq_length: 512


In [30]:
def count_tokens_in_csv(df):
    tokenizer = model.tokenizer 

    df["token_count"] = df["utterance"].astype(str).apply(
        lambda x: len(tokenizer.encode(x, add_special_tokens=True))
    )
    
    return df

In [31]:
df_with_tokens = count_tokens_in_csv(dataset_filtered_politycy)
print(df_with_tokens.head())

Token indices sequence length is longer than the specified maximum sequence length for this model (3677 > 512). Running this sequence through the model will result in indexing errors


                                         politician  \
0  Prezydent Rzeczypospolitej Polskiej Andrzej Duda   
1                 Sekretarz Poseł Bożena Żelazowska   
2            Sekretarz Poseł Alicja Łepkowska-Gołaś   
3                 Sekretarz Poseł Bożena Żelazowska   
4            Sekretarz Poseł Alicja Łepkowska-Gołaś   

                                           utterance  sejm_session  \
0  Drodzy Rodacy! Wielce Szanowny Panie Marszałku...             1   
1  (Sekretarz poseł odczytuje nazwiska posłów)Pos...             1   
2  Poseł Andrzej Adamczyk: Ślubuję. Tak mi dopomó...             1   
3  Poseł Agnieszka Pomaska: Ślubuję.Poseł Cezary ...             1   
4  Poseł Mirosław Suchoń: Ślubuję. Tak mi dopomóż...             1   

         date   political_club  token_count  
0  2023-11-13  Nieznana partia         3677  
1  2023-11-13           PSL-TD          241  
2  2023-11-13               KO          247  
3  2023-11-13           PSL-TD          194  
4  2023-11-13     

In [37]:
df_with_tokens[df_with_tokens["token_count"] < 600].sort_values("token_count")

,politician,utterance,sejm_session,date,political_club,token_count
6952,Członek Państwowej Komisji do spraw Przeciwdzi...,Ślubuję. (Oklaski),13,2024-06-14,Nieznana partia,9
7697,Poseł Piotr Paweł Strach,W imieniu wnioskodawców chciałbym wycofać 6. w...,15,2024-07-12,Polska2050-TD,15
24,Sekretarz Poseł Alicja Łepkowska-Gołaś,Poseł Bożena Żelazowska: Ślubuję. Tak mi dopom...,1,2023-11-13,KO,16
1883,Poseł Grzegorz Adam Płaczek,Panie Marszałku! Wysoka Izbo! Chciałem państwu...,1,2023-12-21,Konfederacja,18
5353,Sekretarz Stanu w Ministerstwie Rolnictwa i Ro...,"Zboże, o którym mowa, było zbożem polskim, pol...",10,2024-04-26,KO,19
...,...,...,...,...,...,...
303,Poseł Paweł Śliz,Pani Marszałek! Wysoka Izbo! Sparafrazuję słow...,1,2023-11-28,Polska2050-TD,598
12360,"Sekretarz Stanu w Ministerstwie Rodziny, Pracy...",Szanowna Pani Marszałkini! Szanowni Państwo! W...,22,2024-11-20,KO,599
6321,Poseł Maja Ewa Nowak,Szanowna Pani Marszałek! Wysoka Izbo! Wdrożeni...,12,2024-05-22,Polska2050-TD,599
5168,Poseł Joanna Kluzik-Rostkowska,Pani Marszałek! Wysoka Izbo! Panie Ministrze! ...,10,2024-04-25,KO,599


In [38]:
politycy_phrases_embeddings = df_with_tokens[df_with_tokens["token_count"] < 600]
print(len(politycy_phrases_embeddings))
print(politycy_phrases_embeddings.sample())

12674
               politician                                          utterance  \
4123  Poseł Anna Gembicka  Dziękuję.Panie Marszałku! Wysoka Izbo! W czasa...   

      sejm_session        date political_club  token_count  
4123             8  2024-03-20            PiS          196  


## Konwersja dataframe'u z wypowiedziami na embeddings
wykonanie "get_embedding" na każdej wypowiedzi z dataframe

In [39]:
print("max_seq_length:", model.max_seq_length)

max_seq_length: 512


In [40]:
from tqdm import tqdm

politycy_phrases_embeddings = [
    (get_embedding(row["utterance"]), row["politician"], row["utterance"], row['political_club']) 
    for _, row in tqdm(politycy_phrases_embeddings.iterrows(), total=len(politycy_phrases_embeddings))
]

df_politycy_phrases_embeddings = pd.DataFrame(politycy_phrases_embeddings, columns=["embedding", "politician", "utterance", 'political_club'])
print(df_politycy_phrases_embeddings.shape)

data = []
for idx, row in df_politycy_phrases_embeddings.iterrows():
    embedding = row['embedding']  # Dostęp do kolumny 'embedding'
    politician = row['politician']
    utterance = row['utterance']
    political_club = row['political_club']
    
    data.append({
        "embedding": ','.join(map(str, embedding)),  # Zapis embeddingu jako string
        "politician": politician,
        "utterance": utterance,
        "political_club": political_club
    })

df_politycy_phrases_embeddings = pd.DataFrame(data)
#df_politycy_phrases_embeddings.to_excel('politycy_embeddings_stsb.xlsx', index=False)
#df_politycy_phrases_embeddings.to_excel('politycy_embeddings_minilm12.xlsx', index=False)
df_politycy_phrases_embeddings.to_excel('politycy_embeddings_polbert.xlsx', index=False)

100%|██████████| 12674/12674 [48:12<00:00,  4.38it/s] 


(12674, 4)


In [41]:
len(df_politycy_phrases_embeddings)

12674

In [22]:
df_politycy_phrases_embeddings.head(5)

,embedding,politician,utterance,political_club
0,"[-0.13537993, 0.3929176, 0.02940008, -0.283716...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Ekscelencjo, Czcigodny Księże Arcybiskupie, Dz...",Nieznana partia
1,"[0.06613037, 0.33783805, 0.32265285, 0.0248571...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Dziękuję, dziękuję i jeszcze raz dziękuję wszy...",Nieznana partia
2,"[-0.010987518, 0.08707595, -0.072819464, 0.147...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Oczywiście my, Polacy, w wielu sprawach istotn...",Nieznana partia
3,"[-0.13236119, 0.27273113, -0.3124308, -0.14640...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Najważniejsze jednak jest to, że spory te rozs...",Nieznana partia
4,"[-0.34609145, 0.33584738, -0.019541811, -0.048...",Prezydent Rzeczypospolitej Polskiej Andrzej Duda,"Polacy ponownie uwierzyli, że ich głos ma znac...",Nieznana partia


# Porównywanie wypowiedzi użytkownika z politykami

## Przygotowanie datasetu z excela
Załadowanie datasetu z wypowiedziami polityków z rozszerzeniem .csv oraz dokonanie konwersji na obiekt typu array na kolumnie "embedding"

In [4]:
df_politycy_phrases_embeddings = pd.read_csv('politycy_embeddings_polbert.csv', encoding='utf-8', sep=';')
df_politycy_phrases_embeddings = df_politycy_phrases_embeddings.dropna(subset=["embedding", "politician", "utterance", 'political_club'])

In [5]:
df_politycy_phrases_embeddings.sample()

,embedding,politician,utterance,political_club
2082,"0.54001087,0.31523195,0.2024596,-0.42209664,-0...",Poseł Dorota Olko,Pani Marszałek! Wysoka Izbo! Mówiliśmy dzisiaj...,Lewica


In [6]:
df_politycy_phrases_embeddings["embedding"] = df_politycy_phrases_embeddings["embedding"].apply(lambda x: np.array(list(map(float, x.split(',')))))

In [7]:
print(f"Liczba wierszy: {len(df_politycy_phrases_embeddings)}")
print(f"Kształt DataFrame: {df_politycy_phrases_embeddings.shape}")
display(df_politycy_phrases_embeddings.sample())

Liczba wierszy: 12674
Kształt DataFrame: (12674, 4)


,embedding,politician,utterance,political_club
2441,"[0.41571304, -0.07338481, 0.32779106, -0.08391...",Poseł Tomasz Piotr Nowak,Panie Marszałku! Panie Ministrze! Wysoka Izbo!...,KO


## Porównywanie

In [11]:
#user_statement = "Zielona energia jest bardzo istotna i powinniśmy skupić się na rozwoju rozwiązaniach ekologicznych. Musimy redukować energię węglową i zastapić ją energią z wiatru czy z wody."
#user_statement = "Polacy potrzebują legalnej aborcji, ponieważ to jest prawo kobiety i nie można przymuszać kobiet."
#user_statement = "Aborcja to zabójstwo dziecka i nie można tego zalegalizować. Jest to sprzeczne z jakimikolwiek prawami moralnymi i religijnymi"
#user_statement = "Aborcja powinna być dozwolona w przypadku gwaltu, ciężkiej choroby dziecka lub możliwej śmierci matki. Ale nigdy nie dozwolona na życzenie."
#user_statement = "Prawo do posiadania broni powinno być dozwolone dla przeciętnego obywatela. Ludzie muszą mieć możliwość obrony wlasnej."
user_statement = "Polska powinna wspierać Ukrainę w tej wojnie i zagwarantować jej wszelką pomoc finansową i militarną."

#avg_similarity, top_similarities = get_similarity(user_input, df_politycy_phrases_embeddings)
similiar_results = get_similarity_politician(user_statement, df_politycy_phrases_embeddings)

print(user_statement)
for similarity, politician, political_club, utterance in similiar_results:
    print(f"Polityk: {politician}, Partia: {political_club}, Podobieństwo: {similarity}, \n Wypowiedź: {utterance} \n")

Polska powinna wspierać Ukrainę w tej wojnie i zagwarantować jej wszelką pomoc finansową i militarną.
Polityk: Poseł Klaudia Jachira, Partia: KO, Podobieństwo: 0.7589058923180627, 
 Wypowiedź: Dziękuję.Panie Marszałku! Wysoka Izbo! Unia Europejska nie powinna robić interesów z Rosją, tak jak alianci nie wspierali III Rzeszy podczas II wojny światowej. Z tego powodu, że każdy deal obróciłby się przeciwko nim. Rosja jest globalnym zagrożeniem dla świata i wszystkie środki najpierw wykorzysta do wzmocnienia swojej siły militarnej.Jestem jak najbardziej za zakazem importu produktów rolnych z Białorusi i Rosji, a jednocześnie martwi mnie fakt, że wszelkie nałożone już unijne sankcje, jak te dotyczące półprzewodników i kluczowych technologii, są niestety przez Rosję obchodzone. Trafiają one tam poprzez inne kraje, takie jak Turcja, Zjednoczone Emiraty Arabskie czy Kazachstan. Oficjalnie jednak wszyscy zaprzeczają, jakoby pomagali Moskwie.Uważam, że Unia Europejska powinna dołożyć więcej star